In [ ]:
import pandas as pd

# Часть 1

In [ ]:
def load_csv(path: str, sep: str = ";") -> pd.DataFrame:
    df = pd.read_csv(path, sep=sep)
    return df

def load(path: str, sep: str = ";") -> pd.DataFrame:
    if path.split(".")